In [61]:
from indicnlp.tokenize import sentence_tokenize as sentence_tokenize_indic
import hasami

source_lang = "en"
target_lang = "zh"

sentencizer_source_language = "english"
sentencizer_target_language = "japanese"

split = "test"

domains = ["auto", "education", "network", "phone"]  
domain = domains[3]

In [62]:
from fine_grained_tech.data_loader import load_data

source_texts, target_texts, _, _ = load_data(source_lang, target_lang, split, domain, None)


In [63]:
from nltk import tokenize

def split_into_sentences(text, sentencizer_language):
    if sentencizer_language == "hindi":
        sent_tokenize = lambda text : sentence_tokenize_indic.sentence_split(text, "hi")
    elif sentencizer_language == "japanese":
        sent_tokenize = hasami.segment_sentences
    else:
        sent_tokenize = lambda text : tokenize.sent_tokenize(text, sentencizer_language)
    if isinstance(text, list):
        sentencized_examples = []
        num_sentences = None
        for example in text:
            sentences = sent_tokenize(example)
            if isinstance(sentences, str):
                sentences = [sentences]
            sentencized_examples.append(sentences)
            if num_sentences is None:
                num_sentences = len(sentences)
            elif num_sentences != len(sentences):
                print("ERROR: Number of sentences in examples is not consistent.")
                print(text)
    
        return [[examples[i] for examples in sentencized_examples] for i in range(len(sentencized_examples[0]))]
    else:
        return sent_tokenize(text)


In [64]:
source_texts_new, target_texts_new = [], []
bad_examples = []
a, b = 0,0
for source, target in zip(source_texts, target_texts):
    source_sentences = split_into_sentences(source, sentencizer_source_language)
    target_sentences = split_into_sentences(target, sentencizer_target_language)
    
    if len(source_sentences) == len(target_sentences):
        source_texts_new.extend(source_sentences)
        target_texts_new.extend(target_sentences)
    else:
        if len(source_sentences) > len(target_sentences):
            a += 1
        else:
            b += 1
        bad_examples.append((source_sentences, target_sentences))
        print("Different number of sentences found in the following example:")
        print(source_sentences)
        print(target_sentences)
        print()

print("Number of bad examples:", len(bad_examples))
print("Number of examples with more source sentences:", a)
print("Number of examples with more target sentences:", b)

Different number of sentences found in the following example:
['Develop applications with various resolutions, horizontal screens, large screens, and the vertical screens of small mobile phones.', 'And various kinds of interfaces can automatically adapt to each other, which makes it more convenient for development.']
['开发多种分辨率，横屏应用，大屏应用，和你竖屏的小手机的应用，多种界面能够自动适配，能够更方便地开发。']

Different number of sentences found in the following example:
['We found that there tends to be a big loss rate from the time when a user finds an app to download it, because he is not willing to wait that long.', 'Also, the download takes time and needs network.']
['我们发现从一个用户找到一个应用，再到下载往往有很大的损失率，因为他不愿意再等待那个时间，下载要花时间，要用网络。']

Different number of sentences found in the following example:
['We intelligently offer recommendations to our users based on the excellent service quality of mobile atomic services.', 'This is what we want to do in accelerating the intelligentization of the digital world.']
['我们根据手机原子服务优秀的服务质量，来智

In [65]:
# write split texts into files
source_path = f"./fine_grained_tech/FDMT3.0/{split}/{domain}-sentences.{split}.{source_lang}"
target_path = f"./fine_grained_tech/FDMT3.0/{split}/{domain}-sentences.{split}.{target_lang}"

with open(source_path, "w") as f:
    for example in source_texts_new:
        if isinstance(example, list):
            f.write("\n".join(example) + "\n")
        else:
            f.write(example + "\n")
with open(target_path, "w") as f:
    for example in target_texts_new:
        if isinstance(example, list):
            f.write(example[0] + "\n")
        else:
            f.write(example + "\n")
